## `ConvLSTM` Evaluation.
* The aim of this notebook is to evaluate trained `ConvLSTM` models.

* 2 evaluations take place in this NB, the first is an evaluation of the metrics on the entire sequence and the second is metrics `CSI` and `LPIPS` / time.

* The `ConvLSTM` models in this NB are roll-out or next-step models. Please see `convlstm_inference_roll_out.ipynb` for more details. A single-shot ConvLSTM was experimented with in `convlstm_inference_one_shot.ipynb` but to make the sampling method more aligned to DYffusion, the roll-out was chosen.

In [1]:
import os
from pathlib import Path
from typing import Any, Dict, List, Tuple

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import xarray as xr
import xskillscore as xs
from livelossplot import PlotLosses
from matplotlib.colors import ListedColormap
from scipy import io
from torch.nn import L1Loss, MSELoss
from torch.utils.data import DataLoader
from torchmetrics.image import LearnedPerceptualImagePatchSimilarity as LPIPS
from torchmetrics.regression import CriticalSuccessIndex
from tqdm import tqdm

from rainnow.src.conv_lstm_utils import (
    IMERGDataset,
    create_eval_loader,
    plot_predicted_sequence,
    save_checkpoint,
    train,
    validate,
)
from rainnow.src.loss import CBLoss, LPIPSMSELoss
from rainnow.src.models.conv_lstm import ConvLSTMModel
from rainnow.src.normalise import PreProcess
from rainnow.src.utilities.loading import load_imerg_datamodule_from_config
from rainnow.src.utilities.utils import (
    get_device,
    transform_0_1_to_minus1_1,
    transform_minus1_1_to_0_1,
)

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/tensordict/_pytree.py:147: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  register_pytree_node(


#### `helpers`

In [4]:
# ** DIR helpers **
BASE_PATH = "/teamspace/studios/this_studio"

CKPT_BASE_PATH = f"{BASE_PATH}/DYffcast/rainnow/results/"
CONFIGS_BASE_PATH = f"{BASE_PATH}/DYffcast/rainnow/src/dyffusion/configs/"

CKPT_DIR = "checkpoints"
CKPT_CFG_NAME = "hparams.yaml"
DATAMODULE_CONFIG_NAME = "imerg_precipitation.yaml"
# whether or not to get last.ckpt or to get the "best model" ckpt (the other one in the folder).
GET_LAST = False

# ** Dataloader Params **
BATCH_SIZE = 12
NUM_WORKERS = 0

INPUT_SEQUENCE_LENGTH = 4
OUTPUT_SEQUENCE_LENGTH = 1

# ** plotting helpers **
# cmap = io.loadmat("../../src/utilities/cmaps/colormap.mat")
cmap = io.loadmat(f"{BASE_PATH}/DYffcast/rainnow/src/utilities/cmaps/colormap.mat")
rain_cmap = ListedColormap(cmap["Cmap_rain"])
global_params = {"font.size": 8}  # , "font.family": "Times New Roman"}
plt_params = {"wspace": 0.1, "hspace": 0.15}
ylabel_params = {"ha": "right", "va": "bottom", "labelpad": 1, "fontsize": 7.5}

# ** get device **
device = get_device()

Cuda installed! Running on GPU! (device = cuda)


#### `Instantiate + Load in the datamodule`

In [5]:
datamodule = load_imerg_datamodule_from_config(
    cfg_base_path=CONFIGS_BASE_PATH,
    cfg_name=DATAMODULE_CONFIG_NAME,
    overrides={
        "boxes": ["0,0", "1,0", "2,0", "2,1"],
        "window": 1,
        "horizon": 8,
        "prediction_horizon": 8,
        "sequence_dt": 1,
    },
)

datamodule.setup("test")

[2024-11-28 09:34:37][imerg_precipitation.py][INFO] --> training, validation & test using 4 (i, j) boxes: ['0,0', '1,0', '2,0', '2,1'].
[2024-11-28 09:34:37][imerg_precipitation.py][INFO] --> test data split: [202307010000, 202401010000]


[2024-11-28 09:34:41][torch_datasets.py][INFO] --> creating TEST tensor dataset.
[2024-11-28 09:34:41][normalise.py][INFO] --> pprocessing w/ percentiles (1st, 99th): [0.0, 5.670000076293945],  (min, max): [0.0, 3.23434630590838]
[2024-11-28 09:34:42][abstract_datamodule.py][INFO] -->  Dataset test size: 979


#### `Create the test_dataset`

In [6]:
# create the datasets.
test_dataset = IMERGDataset(
    datamodule, "test", sequence_length=INPUT_SEQUENCE_LENGTH, target_length=OUTPUT_SEQUENCE_LENGTH
)

test_loader = DataLoader(
    dataset=test_dataset, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS, shuffle=False
)

#### `Instantiate the preprocessor object`

In [7]:
# ** instantiate the preprocesser obj **
pprocessor = PreProcess(
    percentiles=datamodule.normalization_hparams["percentiles"],
    minmax=datamodule.normalization_hparams["min_max"],
)

[2024-11-28 09:34:42][normalise.py][INFO] --> pprocessing w/ percentiles (1st, 99th): [0.0, 5.670000076293945],  (min, max): [0.0, 3.23434630590838]


#### `Get Metrics`

In [8]:
# instantiate metrics.
lpips = LPIPS(reduction="mean", normalize=True).to(
    device
)  # set to True so that the function normalises to [-1, 1].
mse = MSELoss(reduction="mean")
csi_nodes = [2, 10, 18]
# need to get the nodes to the same scale as the data. See NB:  imerg_rainfall_classes.ipynb for rain classes + distributions.
normed_csi_nodes = pprocessor.apply_preprocessing(np.array(csi_nodes))
csi2 = CriticalSuccessIndex(threshold=normed_csi_nodes[0]).to(device)
csi10 = CriticalSuccessIndex(threshold=normed_csi_nodes[1]).to(device)
csi18 = CriticalSuccessIndex(threshold=normed_csi_nodes[-1]).to(device)

#### `Evaluation Metrics (entire predictions)`

In [9]:
# ** create the eval dataloader **
eval_loader, _ = create_eval_loader(
    data_loader=test_loader, horizon=8, input_sequence_length=4, img_dims=(128, 128)
)

** eval loader (INFO) **
Num samples = 1221 w/ dims: torch.Size([12, 1, 128, 128])



In [10]:
# ConvLSTM params (make sure that they match up with the model checkpoint).
KERNEL_SIZE = (5, 5)
INPUT_DIMS = (1, 128, 128)  # C, H, W
OUTPUT_CHANNELS = 1
HIDDEN_CHANNELS = [128, 128]
NUM_LAYERS = 2
CELL_DROPOUT = 0.15

In [13]:
# conv lstm models to evaluate.
conv_lstm_models = {
    # ** LCB Loss **
    "convlstm-abcd1234": (
        "I4:T1 | hs=(128, 128), ks=(5, 5), dp=.15 [20E, lr=3e-4] BS=12 LPIPS.",
        nn.Tanh(),
    ),
    "conv_lstm_f1dlb0m7": (
        "I4:T1 | hs=(128, 128), ks=(5, 5), dp=.4 [30E, lr=7e-5] BS=12 reversal=15% LPIPS.",
        nn.Tanh(),
    ),
    "conv_lstm_9h86gnt5": (
        "I4:T1 | hs=(128, 128), ks=(5, 5), dp=.40 [20E, lr=7e-5] BS=12 reveral=20% LPIPS.",
        nn.Tanh(),
    ),
    "conv_lstm_wwj6eryz": (
        "I4:T1 | hs=(128, 128), ks=(5, 5), dp=.40 [10E, lr=7e-5] BS=12 reveral=20% LPIPS.",
        nn.Tanh(),
    ),
    "conv_lstm_i93g6pzb": (
        "I4:T1 | hs=(128, 128), ks=(5, 5), dp=.45 [10E, lr=3e-4] BS=24 reveral=20% LPIPS.",
        nn.Tanh(),
    ),
    # ** BCE Loss **
    "convlstm-a8kwo8jx": (
        "I4:T1 | hs=(128, 128), ks=(5, 5), dp=.15 [20E, lr=3e-4] BS=12 BCE.",
        nn.Sigmoid(),
    ),
    "conv_lstm_k34y14il": (
        "I4:T1 | hs=(128, 128), ks=(5, 5), dp=.4 [10E, lr=7e-5] BS=12 BCE.",
        nn.Sigmoid(),
    ),
}

In [14]:
# ** evaluate 101 **
eval_metrics = {}
for ckpt_id, model_desc in conv_lstm_models.items():
    # create the model ckpt path.
    # ckpt_id_path = Path(os.path.join(CKPT_BASE_PATH, "", ckpt_id, "checkpoints", f"{ckpt_id}.pt"))
    ckpt_id_path = Path(os.path.join(BASE_PATH, f"{ckpt_id}.pt"))
    print(f"Loading model ckpt from {ckpt_id_path}.")

    OUTPUT_ACTIVATION = model_desc[-1]
    print(f"Instantiating model w/ output activation = {str(model_desc[-1])}.")

    # instantiate a new ConvLSTM model.
    model = ConvLSTMModel(
        input_sequence_length=INPUT_SEQUENCE_LENGTH,
        output_sequence_length=OUTPUT_SEQUENCE_LENGTH,
        input_dims=INPUT_DIMS,
        hidden_channels=HIDDEN_CHANNELS,
        output_channels=OUTPUT_CHANNELS,
        num_layers=NUM_LAYERS,
        kernel_size=KERNEL_SIZE,
        output_activation=OUTPUT_ACTIVATION,
        apply_batchnorm=True,
        cell_dropout=CELL_DROPOUT,
        bias=True,
        device=device,
    )
    model = model.to(device)

    # load in the checkpoint + set to eval() mode.
    model.load_state_dict(
        state_dict=torch.load(ckpt_id_path, map_location=torch.device(device))["model_state_dict"]
    )
    model.eval()

    # ** get preds / target pairs **
    # loop through the custom eval_loader and get the predictions and targets for each X, target pair.
    # at the end of this loop, you have a results list that contains [target, predictions] pairs.
    with torch.no_grad():
        results = []
        for e, (X, target) in tqdm(
            enumerate(eval_loader), total=len(eval_loader), desc=f"Evaluating model {ckpt_id}"
        ):  # enumerate(eval_loader):
            predictions = {}
            _input = X.clone().unsqueeze(0).to(device)
            for t in range(target.size(0)):
                pred = model(_input)  # predict t+1
                if isinstance(model.output_activation, nn.Tanh):
                    pred = transform_minus1_1_to_0_1(pred)

                # add t+i to the predictions.
                predictions[f"t{t+1}"] = pred.squeeze(0)
                # update the inputs with the last pred (auto-regressive rollout)
                _input = torch.concat([_input[:, 1:, ...], pred], dim=1)

            results.append([target, predictions])

        # ** calculate metrics for each preds / target pair **
        # reset metrics for eack ckpt id.
        # overall metrics for target and prediction.
        l2_score = 0
        lpips_score = 0
        csi2_score = 0  # low rain.
        csi10_score = 0  # mid rain.
        csi18_score = 0  # heavy rain.
        for targets, predictions in results:
            # concat to get entire sequence.
            pred_seq = torch.cat([v for _, v in predictions.items()], dim=0)

            # get metrics.
            l2_score += mse(pred_seq.to(device), targets.to(device))
            # lpips score. Inputs need to have 3 channels.
            lpips_score += lpips(
                torch.clamp(pred_seq.expand(-1, 3, -1, -1), 0, 1).to(device),
                torch.clamp(targets.expand(-1, 3, -1, -1), 0, 1).to(device),
            )
            # csi score at different thresholds.
            csi2_score += csi2(pred_seq.to(device), targets.to(device))
            csi10_score += csi10(pred_seq.to(device), targets.to(device))
            csi18_score += csi18(pred_seq.to(device), targets.to(device))

        eval_metrics[ckpt_id] = {
            "MSE": l2_score.item() / len(eval_loader),
            "lpips": lpips_score.item() / len(eval_loader),
            "csi2": csi2_score.item() / len(eval_loader),
            "csi10": csi10_score.item() / len(eval_loader),
            "csi18": csi18_score.item() / len(eval_loader),
        }

# create df, format it and export it to a .csv.
df_results = pd.DataFrame(eval_metrics).T
df_results[["MSE", "lpips", "csi2", "csi10", "csi18"]]

Loading model ckpt from /teamspace/studios/this_studio/convlstm-abcd1234.pt.
Instantiating model w/ output activation = Tanh().


Evaluating model convlstm-abcd1234: 100%|██████████| 1221/1221 [03:48<00:00,  5.34it/s]


Loading model ckpt from /teamspace/studios/this_studio/conv_lstm_f1dlb0m7.pt.
Instantiating model w/ output activation = Tanh().


Evaluating model conv_lstm_f1dlb0m7: 100%|██████████| 1221/1221 [03:48<00:00,  5.34it/s]


Loading model ckpt from /teamspace/studios/this_studio/conv_lstm_9h86gnt5.pt.
Instantiating model w/ output activation = Tanh().


...

Loading model ckpt from /teamspace/studios/this_studio/convlstm-a8kwo8jx.pt.
Instantiating model w/ output activation = Sigmoid().


Evaluating model convlstm-a8kwo8jx: 100%|██████████| 1221/1221 [03:46<00:00,  5.39it/s]


Loading model ckpt from /teamspace/studios/this_studio/conv_lstm_k34y14il.pt.
Instantiating model w/ output activation = Sigmoid().


Evaluating model conv_lstm_k34y14il: 100%|██████████| 1221/1221 [03:43<00:00,  5.45it/s]


,MSE,lpips,csi2,csi10,csi18
convlstm-abcd1234,0.008515,0.272168,0.139340,0.027360,0.010332
conv_lstm_f1dlb0m7,0.043557,0.363798,0.141904,0.022638,0.008237
conv_lstm_9h86gnt5,0.011890,0.335648,0.132495,0.030015,0.010908
conv_lstm_wwj6eryz,0.015996,0.306106,0.145382,0.028931,0.010627
conv_lstm_i93g6pzb,0.015239,0.323197,0.149694,0.030855,0.011330
convlstm-a8kwo8jx,0.006830,0.381121,0.145322,0.033976,0.012780
conv_lstm_k34y14il,0.005175,0.334931,0.164663,0.037835,0.015641


#### `Eval Metrics (CSI + LPIPS / t)` 

In [15]:
eval_metrics_per_t = {}
for ckpt_id, model_desc in conv_lstm_models.items():
    # create the model ckpt path.
    # ckpt_id_path = Path(os.path.join(CKPT_BASE_PATH, "", ckpt_id, "checkpoints", f"{ckpt_id}.pt"))
    ckpt_id_path = Path(os.path.join(BASE_PATH, f"{ckpt_id}.pt"))
    print(f"Loading model ckpt from {ckpt_id_path}.")

    OUTPUT_ACTIVATION = model_desc[-1]
    print(f"Instantiating model w/ output activation = {str(model_desc[-1])}.")

    # instantiate a new ConvLSTM model.
    model = ConvLSTMModel(
        input_sequence_length=INPUT_SEQUENCE_LENGTH,
        output_sequence_length=OUTPUT_SEQUENCE_LENGTH,
        input_dims=INPUT_DIMS,
        hidden_channels=HIDDEN_CHANNELS,
        output_channels=OUTPUT_CHANNELS,
        num_layers=NUM_LAYERS,
        kernel_size=KERNEL_SIZE,
        output_activation=OUTPUT_ACTIVATION,
        apply_batchnorm=True,
        cell_dropout=CELL_DROPOUT,
        bias=True,
        device=device,
    )
    model = model.to(device)

    # load in the checkpoint + set to eval() mode.
    model.load_state_dict(
        state_dict=torch.load(ckpt_id_path, map_location=torch.device(device))["model_state_dict"]
    )
    model.eval()

    # ** get preds / target pairs **
    # loop through the custom eval_loader and get the predictions and targets for each X, target pair.
    # at the end of this loop, you have a results list that contains [target, predictions] pairs.
    with torch.no_grad():
        results = []
        for e, (X, target) in tqdm(
            enumerate(eval_loader), total=len(eval_loader), desc=f"Evaluating model {ckpt_id}"
        ):  # enumerate(eval_loader):
            predictions = {}
            _input = X.clone().unsqueeze(0).to(device)
            for t in range(target.size(0)):
                pred = model(_input)  # predict t+1
                if isinstance(model.output_activation, nn.Tanh):
                    pred = transform_minus1_1_to_0_1(pred)

                # add t+i to the predictions.
                predictions[f"t{t+1}"] = pred.squeeze(0)
                # update the inputs with the last pred (auto-regressive rollout)
                _input = torch.concat([_input[:, 1:, ...], pred], dim=1)

            results.append([target, predictions])

        # create csi stores.
        csi2_score_t = torch.zeros(target.size(0)).to(device)
        csi10_score_t = torch.zeros(target.size(0)).to(device)
        csi18_score_t = torch.zeros(target.size(0)).to(device)

        # perceptual loss scores.
        lpips_score_t = torch.zeros(target.size(0)).to(device)

        for targets, predictions in results:
            for e, (k, v) in enumerate(predictions.items()):
                # loop through all the ts and compute the relevant CSI scores.
                csi2_score_t[e] += csi2(targets[e].to(device), v[0, ...].to(device))
                csi10_score_t[e] += csi10(targets[e].to(device), v[0, ...].to(device))
                csi18_score_t[e] += csi18(targets[e].to(device), v[0, ...].to(device))

                lpips_score_t[e] += lpips(
                    torch.clamp(targets[e].expand(1, 3, -1, -1), 0, 1).to(device),
                    torch.clamp(v[0, ...].expand(1, 3, -1, -1), 0, 1).to(device),
                )

        # normalise the scores.
        eval_metrics_per_t[ckpt_id] = {
            "csi2_t": csi2_score_t / len(eval_loader),
            "csi10_t": csi10_score_t / len(eval_loader),
            "csi18_t": csi18_score_t / len(eval_loader),
            "lpips_t": lpips_score_t / len(eval_loader),
        }

Loading model ckpt from /teamspace/studios/this_studio/convlstm-abcd1234.pt.
Instantiating model w/ output activation = Tanh().


Evaluating model convlstm-abcd1234: 100%|██████████| 1221/1221 [03:43<00:00,  5.47it/s]


Loading model ckpt from /teamspace/studios/this_studio/conv_lstm_f1dlb0m7.pt.
Instantiating model w/ output activation = Tanh().


Evaluating model conv_lstm_f1dlb0m7: 100%|██████████| 1221/1221 [03:41<00:00,  5.52it/s]


Loading model ckpt from /teamspace/studios/this_studio/conv_lstm_9h86gnt5.pt.
Instantiating model w/ output activation = Tanh().


...

Evaluating model conv_lstm_i93g6pzb: 100%|██████████| 1221/1221 [03:37<00:00,  5.61it/s]


Loading model ckpt from /teamspace/studios/this_studio/convlstm-a8kwo8jx.pt.
Instantiating model w/ output activation = Sigmoid().


Evaluating model convlstm-a8kwo8jx: 100%|██████████| 1221/1221 [03:46<00:00,  5.40it/s]


Loading model ckpt from /teamspace/studios/this_studio/conv_lstm_k34y14il.pt.
Instantiating model w/ output activation = Sigmoid().


Evaluating model conv_lstm_k34y14il: 100%|██████████| 1221/1221 [03:51<00:00,  5.27it/s]


In [16]:
horizon = 8

# get csi + lpips dfs.
df_all = {}
for k, v in eval_metrics_per_t.items():
    dfs = {}
    for metric, values in v.items():
        dfs[metric] = pd.DataFrame(
            data=[[i.item() for i in values]], columns=[f"t{i+1}" for i in range(len(values))]
        )
    df_all[k] = dfs

df_csi2 = pd.DataFrame(columns=[f"t{i+1}" for i in range(horizon)])
df_csi10 = pd.DataFrame(columns=[f"t{i+1}" for i in range(horizon)])
df_csi18 = pd.DataFrame(columns=[f"t{i+1}" for i in range(horizon)])
df_lpips = pd.DataFrame(columns=[f"t{i+1}" for i in range(horizon)])

for model_name, metrics in df_all.items():
    model_name_clean = model_name.rsplit(".", 1)[0]

    csi2_values = metrics["csi2_t"].iloc[0].values
    csi10_values = metrics["csi10_t"].iloc[0].values
    csi18_values = metrics["csi18_t"].iloc[0].values
    lpips_values = metrics["lpips_t"].iloc[0].values

    df_csi2.loc[model_name_clean] = csi2_values
    df_csi10.loc[model_name_clean] = csi10_values
    df_csi18.loc[model_name_clean] = csi18_values
    df_lpips.loc[model_name_clean] = lpips_values

In [17]:
df_csi2

,t1,t2,t3,t4,t5,t6,t7,t8
convlstm-abcd1234,0.409545,0.256894,0.177189,0.132172,0.101844,0.088881,0.079038,0.070007
conv_lstm_f1dlb0m7,0.395652,0.264934,0.211360,0.161144,0.125066,0.107537,0.092847,0.083068
conv_lstm_9h86gnt5,0.343507,0.223668,0.170193,0.127173,0.093653,0.078596,0.066404,0.056264
conv_lstm_wwj6eryz,0.346521,0.237239,0.200770,0.152150,0.115731,0.100271,0.085675,0.074499
conv_lstm_i93g6pzb,0.354462,0.241240,0.204958,0.157583,0.114171,0.099549,0.085553,0.072522
convlstm-a8kwo8jx,0.382280,0.256391,0.184229,0.136087,0.102651,0.085727,0.071334,0.059770
conv_lstm_k34y14il,0.431672,0.293206,0.219680,0.165703,0.127074,0.108157,0.091020,0.077188


In [18]:
df_csi10

,t1,t2,t3,t4,t5,t6,t7,t8
convlstm-abcd1234,0.196756,0.085387,0.047304,0.030812,0.021354,0.016173,0.013719,0.011601
conv_lstm_f1dlb0m7,0.153633,0.063025,0.040696,0.027660,0.019865,0.016451,0.013659,0.011927
conv_lstm_9h86gnt5,0.138102,0.061805,0.040662,0.029726,0.020899,0.016101,0.012788,0.010402
conv_lstm_wwj6eryz,0.138549,0.063507,0.044933,0.031876,0.022694,0.018159,0.014990,0.012627
conv_lstm_i93g6pzb,0.147087,0.064601,0.043774,0.032007,0.022987,0.018818,0.015808,0.012907
convlstm-a8kwo8jx,0.163104,0.081930,0.050999,0.033489,0.023991,0.018729,0.014805,0.011701
conv_lstm_k34y14il,0.203662,0.089252,0.051320,0.031776,0.022753,0.018286,0.014581,0.011754


In [19]:
df_csi18

,t1,t2,t3,t4,t5,t6,t7,t8
convlstm-abcd1234,0.119986,0.045686,0.020388,0.012271,0.008481,0.006303,0.004930,0.004102
conv_lstm_f1dlb0m7,0.082886,0.027547,0.016282,0.010519,0.007398,0.005987,0.004994,0.004269
conv_lstm_9h86gnt5,0.075541,0.025412,0.015594,0.011031,0.007806,0.005767,0.004526,0.003646
conv_lstm_wwj6eryz,0.081394,0.028186,0.017909,0.011986,0.008567,0.006621,0.005343,0.004582
conv_lstm_i93g6pzb,0.086424,0.028502,0.017626,0.011863,0.008592,0.006863,0.005648,0.004532
convlstm-a8kwo8jx,0.092420,0.039859,0.021583,0.012946,0.009399,0.006943,0.005406,0.004080
conv_lstm_k34y14il,0.100361,0.029523,0.013243,0.008495,0.006383,0.005630,0.004465,0.003583


In [20]:
df_lpips

,t1,t2,t3,t4,t5,t6,t7,t8
convlstm-abcd1234,0.112798,0.179003,0.228452,0.269984,0.306517,0.336390,0.362645,0.381964
conv_lstm_f1dlb0m7,0.148410,0.239454,0.302016,0.358847,0.408373,0.449610,0.486470,0.517351
conv_lstm_9h86gnt5,0.209007,0.286848,0.322093,0.341417,0.361709,0.376277,0.390269,0.398401
conv_lstm_wwj6eryz,0.158053,0.238760,0.267362,0.304854,0.340756,0.360060,0.380509,0.398832
conv_lstm_i93g6pzb,0.180151,0.250406,0.281499,0.325177,0.359863,0.378376,0.398438,0.412306
convlstm-a8kwo8jx,0.284755,0.345323,0.376389,0.390589,0.401702,0.410233,0.418553,0.422708
conv_lstm_k34y14il,0.182539,0.272589,0.323240,0.350610,0.370502,0.384011,0.395276,0.401460


### END OF SCRIPT.